# 画像分類の実装

本章では Python の代表的な画像処理のパッケージである OpenCV や Pillow を用いての基礎的な画像データの取り扱い方法について学び、その後 TensorFlow を用いて Convolutional Neural Network (以下 CNN) の実装方法を確認します。

## 本章の構成  

- 画像処理の基礎
- 画像のクラス分類の実装
- CNN モデルの順伝播の流れ


### GPU の設定

本章では Colab 上の Graphics Processing Unit (以下 GPU) を用いてモデルの学習を行います。GPU を使用するために事前に下記の設定を行っておいて下さい。  

1. GPU のランタイムの設定
![GPU 設定 1](http://drive.google.com/uc?export=view&id=1wVi7zFp1vJnOFxVuKiXa0N5mDWB00hFE)

![GPU 設定 2](http://drive.google.com/uc?export=view&id=1p0ftgj0bRjTgm5L6DxSar5XolowHegxj)

2. ランタイムの再起動
  - 「ランタイム」 → 「ランタイムの再起動」を選択肢、ランタイムの再起動を行います。  




## 画像処理の基礎

OpenCV と Pillow という Python の画像処理パッケージを使用しての基礎的な画像の取り扱い方法について学びます。画像処理には[こちら](https://drive.google.com/file/d/1rRPd3wrXmhfk6SPyT2A_sUgy6CrtjPZ_/view?usp=sharing)の画像を使用します。リンク先の画像の上で右クリックから画像の保存を選択して下さい。  

ダウンロード後、Colab にアップロードを行って下さい。  

In [ ]:
from google.colab import files
uploaded = files.upload()

### Pillow の基礎

Pillow は`PIL`という名前で登録されています。モジュールをインポートし、Pillow を用いて画像の読み込みましょう。また、読み込み後み `resize()` メソッドを使用して画像サイズを変更します。  

In [2]:
# 必要なモジュールのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from PIL import Image

In [4]:
# 画像の読み込み
img = Image.open('sample.png')

Pillow を用いて読み込んだ画像は PngImageFile オブジェクトとなります。  
PngImageFile オブジェクトは画像データであるため、Notebook 上で変数を実行すると画像を表示することが可能です。

In [ ]:
img

In [ ]:
# サイズ変更
img.resize((300, 100))

データの型を確認します。

In [ ]:
type(img)

Pillow を用いて簡単な画像データ操作について学びます。  
直接このデータ操作は後ほどの CNN の実装とは関係しませんが、基礎的な処理方法を抑えておきましょう。

#### 画像の回転

PngImageFile オブジェクトは様々なメソッドを持ち、簡単に画像処理を適用することができます。画像の回転には `rotate()` メソッドを使用します。引数に回転の角度を取ります。   
CNN モデルの学習の際に使用するデータにノイズ成分を追加する際などに用います。  



In [ ]:
# 画像の回転
img.rotate(45)

#### 画像のクロップ

画像のクロップは `crop()` メソッドを使用します。引数に x 軸の座標、y 軸の座標、横幅、縦幅をとります。

In [ ]:
# 画像のクロップ
img.crop((0, 0, 150, 150))

#### NumPy の ndarray オブジェクトに変換

TensorFlow を用いてニューラルネットワークを実装する際のデータは NumPy の ndarray オブジェクトである必要がありました。  

変換は単純に `np.array()` クラスを使用します。  

In [10]:
img_array = np.array(img)

In [ ]:
type(img_array)

データの形を確認しましょう。  

In [ ]:
img_array.shape

画像は縦幅 (hegiht) が 400 、横幅 (width) が 300 、チャンネル数 (channel) が 3 となっていることが確認できます。  

それぞれのチャンネルを切り出して、それぞれが red・green・blue であることを確認しましょう。それぞれのチャンネルを切り出し、red のチャンネルを取り出す場合はその他のチャンネルの値を 0 に置き換えます。

In [13]:
img_rgb = img.convert('RGB')
img_array = np.array(img_rgb)
img_array[:, :, 1] *= 0 # blue チャンネルを 0 に
img_array[:, :, 2] *= 0 # green チャンネルを 0 に
img_red = Image.fromarray(img_array) # Pillow の型に変換

In [ ]:
img_red

赤色の画像が取り出すことができました。上記のコードの 0 に置き換えるチャンネルのインデックス番号を変更して、green・blue のチャンネルを抽出した場合もそれぞれ確認しておきましょう。  

Pillow はこの他にも様々な画像データの操作を行うことができます。  
詳細に関してはこちらの[公式ドキュメント](https://pillow.readthedocs.io/en/5.1.x/reference/Image.html)を確認して下さい。  


### OpenCV の基礎

OpenCV は`cv2`という名前で登録されています。Pillow との使用方法は異なりますが、基本的な機能に大きな差異はありません。どちらを使用するかは実際に使用して、使いやすいと思うものを選択して下さい。

In [15]:
import cv2

In [16]:
# 画像の読み込み
img = cv2.imread('sample.png')

OpenCV で読み込んだ画像は NumPy の ndarray 型で読み込まれます。

In [ ]:
# 型を確認
type(img)

In [ ]:
# サイズの確認
img.shape

In [ ]:
# データ型の確認
img.dtype

`uint8`は unsigned int の略であり、符号なし（正の値のみ）の 8 ビット整数であり、0~255 までを表現可能です。  

OpenCV で読み込んだ画像は **BGR** (Blue, Green, Red) の順で格納されているため、画像の描画を行った際には、青みの強い色合いになります。  

画像の描画には matplotlib の `imshow()` 関数を用います。

In [ ]:
plt.imshow(img)

現在 BGR の順で並んでいるチャンネルの配列を RGB に変換します。変換には `cvtColor()` 関数を用います。引数に `COLOR_BGR2RGB` を用い、BGR → RGB への変換を指定しています。

In [21]:
# BGR -> RGB
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(img_rgb)

Pillow と OpenCV の違いをまとめると下記になります。  

|                  | Pillow                | OpenCV          |
| ---------------- | --------------------- | --------------- |
| **オブジェクト** | Pillow - PngImageFile | NumPy - ndarray |
| **チャンネル**     | RGB                   | BGR             |

Pillow と OpenCV の読み込むチャンネルの順番が異なる点には十分注意しましょう。例えば、学習は OpenCV で行い、推論の際は Pillow を使用するようなケースでは、学習時と推論時でチャンネルの順番が異なるため予測結果が望ましくない事が想定されます。

#### グレースケール変換

代表的な画像の前処理の 1 つであるグレースケール変換を施します。こちらも先程使用した `cvtColor()` 関数を使用します。

In [ ]:
# Pillow
img = Image.open('sample.png').convert('L')
img

In [24]:
# OpenCV
img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

matplotlib の`imshow`では、RGBが渡される規定であるため、グレースケールでは変な色合いになってしまいます。描画前に `gray()` 関数を実行しておきましょう。

In [ ]:
plt.imshow(img_gray, cmap='gray')

グレースケール変換を施したことによって チャンネル数が 3 から 1 になっています。データの形を確認しましょう。

In [ ]:
img_gray.shape

#### エッジ検出

画像内の明るさの変化を検出するエッジ検出の実装を行います。エッジ検出のためのフィルタを準備します。フィルタのことを**カーネル (kernel)**とも呼ぶことも覚えておきましょう。

In [27]:
# エッジ検出のフィルタの定義
kernel = np.array([
    [-1, 0, 1],
    [-1, 0, 1],
    [-1, 0, 1]
])

エッジ検出のフィルタを画像に適用します。フィルタを画像に適用することを畳み込み (convolution) とも呼びます。  
畳み込みの演算は `filter2D()` 関数を用います。

In [28]:
img_conv = cv2.filter2D(img_gray, -1, kernel)

エッジ検出フィルタ適用後の画像を確認します。

In [ ]:
plt.imshow(img_conv, cmap='gray')

横方向に対して、エッジが検出できていることが確認できます。縦方向にもエッジ検出のフィルタを適用してみましょう。


In [30]:
kernel = np.array([
    [-1, -1, -1],
    [  0,  0,  0],
    [  1,  1,  1]
])

In [ ]:
img_conv = cv2.filter2D(img_gray, -1, kernel)

In [ ]:
plt.imshow(img_conv, cmap='gray')

先ほどとは異なり、縦方向に輝度の変化量が多い部分が抽出されていることがわかります。特に縦方向の輝度の変化が強い目元を確認すると横方向のエッジ検出のフィルタとの違いが確認することができます。

## 画像のクラス分類の実装

本節では、画像データの基礎的な取り扱い方法を理解した上で、畳み込みニューラルネットワーク (Convolutional Neural Network ; 以下 CNN) の実装を行っていきます。  

今回画像のクラス分類を行う問題設定は 0~9 までの 10 種類の手書き数字になります。使用するデータセットは MNIST と呼ばれるものを使用します。  

![MNIST サンプル](http://drive.google.com/uc?export=view&id=1UN1f-zvpUsnJQOFeJrqSv0or_joH_Gcm)


### データセットの準備

TensorFlow を用いて、CNN を実装する際の画像のデータセットの形式を確認します。画像や自然言語などの非構造化データを取り扱う際にはまず入力値がどのような形式になっているのかを把握することが重要です。  

データセットの読み込みは `tf.keras.datasets.mnist` クラスを用いて取得します。

In [32]:
import tensorflow as tf

GPU が使用可能であるか確認しましょう。  
`name: "/device:GPU:0"` の表示があれば GPU が使用可能な状況となっています。

In [ ]:
# GPU が使用可能であることを確認
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
# データセットの取得
from tensorflow.keras.datasets import mnist
(x_train, t_train), (x_test, t_test) = mnist.load_data()

取得したデータセットはすでに TensorFlow を用いて CNN を実装する際に適したデータ形式となっています。データセットの型、データ型、形などを確認し、どのような形式でデータセットを準備する必要があるのか確認していきます。

In [ ]:
# サンプル数確認
len(x_train), len(t_train), len(x_test), len(t_test)

In [ ]:
# サイズ確認
x_train.shape, t_train.shape, x_test.shape, t_test.shape

In [ ]:
# 1 枚可視化
img = x_train[0]
plt.imshow(img, cmap='gray')

In [ ]:
# 目標値確認
t_train[0]

#### TensorFlow で使用できる形式に変換

 画像データの形を (height, width) から (height, width, channel) へと変換します。また画像データの値の正規化を行います。  
 形の変換は `reshape()` メソッドに変換後の形をタプル型で引数に指定します。  
 正規化は uint8 形式のデータの最大値である 255 で割ることで 0~1 の間に変換します。

In [39]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255.0
x_test = x_test.reshape(10000, 28, 28, 1) / 255.0

In [ ]:
# チャンネルが追加されていることを確認
x_train[0].shape

In [ ]:
# 正規化されていることを確認
x_train[0].min(), x_train[0].max()

最後に入力値は float32 のデータ型に、目標値は int32 のデータ型に変換しておきます。

In [42]:
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
t_train, t_test = t_train.astype('int32'), t_test.astype('int32')

### CNN のモデルの定義

CNN モデルの定義を行います。まず、CNN のモデルの概要を再度確認します。

![CNN モデル](http://drive.google.com/uc?export=view&id=1eDSmSKeLjU-kb-r_F_4JloJPOxrQP0AA)

CNN のモデルは上図のように大きく分けて 3 つの要素からなります。説明に記載されている英字はコードと関連します。  

- 特徴量抽出 : convolution + pooling
  - 画像データからクラス分類などを行う際に使用する特徴量を抽出を行う。
  - 畳み込み (convolution) と縮小 (pooling) を繰り返す。convolution 層を何層追加するのかなどはハイパーパラメータに該当する。
- ベクトル化 : faltten
  - 特徴量抽出後の値をベクトルに変換する。
- 識別 : dense
  - 全結合層、活性化関数を介してクラス分類を行う。  

全体像を把握したところで、モデルの定義を行いましょう。

In [43]:
import os, random

def reset_seed(seed=0):
    os.environ['PYTHONHASHSEED'] = '0'
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
from tensorflow.keras import models,layers

# シードの固定
reset_seed(0)

# モデルのインスタンス化
model = models.Sequential()

# モデルの構築
## 特徴量抽出
model.add(layers.Conv2D(filters=3, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1))) # 畳み込み (convolution) 層
model.add(layers.MaxPooling2D(pool_size=(2, 2))) # pooling 層

## ベクトル化
model.add(layers.Flatten())

## 識別
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

モデルの定義が完了しました。`summary()` メソッドでパラメータを確認します。  

In [ ]:
model.summary()

1 層目の `conv2d` のパラメータの数が $30$ となっています。何故この値なのか確認します。  

- カーネルのサイズ : $3\times3$
- 入力のチャンネル数 : $1$
- 出力のチャンネル数 : $3$
- 重みの数 : $(3\times3)\times1\times3 = 27$
- バイアスの数 : $3$
- 合計のパラメータの数 : $27+3 = 30$

前章で学んだ数学と同じようにパラメータ数があることが確認できました。  注意点として、今回入力値の画像は 1 チャンネルのものを使用していますが、このチャンネル数が 3 になった場合は、重みの数は 3 倍多くなります。  

構造のプロットも行います。

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model)

今回は非常にシンプルな CNN のモデルを定義しました。精度向上のためには、特徴量抽出の部分の convolution 層や pooling 層の数を調整したり、全結合層の層やノードの数を調整します。  

### 目的関数と最適化手法の選択

今回は最適化の手法に Adam を、目的関数は分類の問題設定のため sparse categorical crossentropy を使用します。

In [47]:
# optimizerの設定
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# モデルのコンパイル
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

### モデルの学習

バッチサイズ、エポック数を定義して、モデルの学習を実行します。

In [ ]:
# 学習の実行
history = model.fit(x_train, t_train,
                    batch_size=4096,
                    epochs=30,
                    validation_data=(x_test, t_test))

今回は GPU を使用して学習を行いました。GPU のメモリの使用率は下記の `!nvidia-smi` コマンドを実行します。  

`Memory-Usage` の欄を確認すると`1121MiB / 16280MiB` のように現在どの程度メモリを専有しているか確認できます。  

経験的にバッチサイズはこのメモリを可能な限り使用できる大きさに調整することが多いです。

In [ ]:
!nvidia-smi

### 予測精度の評価

学習結果を確認します。

In [ ]:
results = pd.DataFrame(history.history)
results.tail(3)

In [ ]:
# 損失を可視化
results[['loss', 'val_loss']].plot(title='loss')
plt.xlabel('epochs')

In [ ]:
# 正解率を可視化
results[['accuracy', 'val_accuracy']].plot(title='accuracy')
plt.xlabel('epochs')

損失が下がり、正解率も 95% を超えており、予測精度としては悪くない事が確認できます。続いては実装の中身を分解して確認します。

## CNN モデルの順伝播の流れ

構築した CNN モデルの計算の中身を確認していきます。  
入力画像が特徴抽出からベクトル化にかけてどのように変化しているのかを簡単に確認します。

In [ ]:
# 推論に使用するデータを切り出し + バッチサイズの追加
x_new = np.array([x_train[0]])
x_new.shape

学習済みモデルの層は `layers` 属性から取得することができ、層のインデックス番号を使用すると特定の層の取り出しを行うことが可能です。  

In [ ]:
model.layers

切り出した重みの取得には `get_weights()` メソッドを用います。

In [ ]:
model.layers[0].get_weights()

### convolution 層の計算

切り出した層に値を渡すことによって計算を行うことができます。1 層目の convolution 層の計算を実行し、出力データを画像として可視化してみましょう。  

In [56]:
output = model.layers[0](x_new) # convolution 層の計算
output = output[0].numpy() # NumPy の ndarray オブジェクトに変換

今回の convolution 層のフィルタの数は 3 でした。そのため、出力されるデータのチャンネル数は 3 になります。それぞれのチャンネル毎に可視化を行います。

In [ ]:
output.shape

In [ ]:
# 1 つ目の出力
plt.imshow(output[:, :, 0], cmap='gray')

In [ ]:
# 2 つ目の出力
plt.imshow(output[:, :, 1], cmap='gray')

In [ ]:
# 3 つ目の出力
plt.imshow(output[:, :, 2], cmap='gray')

それぞれ個別のフィルタが適用され、異なる出力が確認できます。この画像から人間側がどのような特徴を抽出しているか理解することは少し困難ですが、前章で学んだ数学の処理が施されている事が確認できます。

### pooling 層の計算

pooling 層の計算を確認します。pooling サイズが 2x2 だったため、出力のサイズは 1/2 になります。  

In [61]:
output = model.layers[0](x_new) # convolution 層の計算
output = model.layers[1](output) # pooling 層の計算（サイズを 1/2 に変換）
output = output[0].numpy()

In [ ]:
output.shape

In [ ]:
# 1 つ目の出力
plt.imshow(output[:, :, 0], cmap='gray')

In [ ]:
# ２つ目の出力
plt.imshow(output[:, :, 1], cmap='gray');

In [ ]:
# 3つ目の出力
plt.imshow(output[:, :, 2], cmap='gray');

### ベクトル化

先程の出力の形は 13, 13, 3 になります。全ての値の数の合計は $13\times13\times3 = 507$ となります。実際に 507 次元のベクトルに変換されていることを確認しましょう。

In [66]:
output = model.layers[0](x_new) # convolution 層の計算
output = model.layers[1](output) # pooling 層の計算（サイズを 1/2 に変換）
output = model.layers[2](output) # ベクトル化
output = output[0].numpy()

In [ ]:
output.shape

## 練習問題 本章のまとめ

本章で学んだ内容を復習しましょう。下記の内容を次のセルに記述し、実行結果を確認してください。（必要に応じてセルの追加を行ってください。）  

CNN モデルのハイパーパラメータ調整を行い、予測精度にどのような変化があるのか確認して下さい。  

*ハイパーパラメータ調整のポイント*

- convolution 層の数
- カーネルサイズ
- パディング
- pooling 層の数
- pooling のサイズ
- バッチノーマリゼーション層の追加
- 全結合層のノード・層の数
- 最適化手法

*発展*  
- 学習済みモデルの convolution 層を切り出し、計算を行い、出力結果を確認して下さい。

In [ ]:
# モデルの定義


In [ ]:
# 目的関数と最適化手法の選択


In [ ]:
# モデルの学習


In [ ]:
# 予測精度の評価


In [ ]:
# convolution 層の出力の確認


---
© 株式会社キカガク及び国立大学法人 豊橋技術科学大学